### imports

In [1]:
import math
import lucene
import pickle
import time
import matplotlib.pyplot as plt
import itertools
import numpy as np
from tqdm import tqdm
from java.io import File
import xml.etree.ElementTree as ET
from collections import defaultdict
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import BytesRefIterator
from org.apache.lucene.index import DirectoryReader, Term
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.analysis.core import WhitespaceAnalyzer
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher, BooleanQuery, BooleanClause, TermQuery, BoostQuery
from org.apache.lucene.search.similarities import BM25Similarity, LMJelinekMercerSimilarity, LMDirichletSimilarity
lucene.initVM()

In [2]:
index_path = './index-enwiki' #index path


directory = FSDirectory.open(File(index_path).toPath())
indexReader = DirectoryReader.open(directory)

In [3]:
a = indexReader.document(1053350)

In [4]:
terms = indexReader.getTermVector(1053350, 'CONTENT')

In [10]:
iterator = terms.iterator()
for term in BytesRefIterator.cast_(iterator):
    t = term.utf8ToString()
    print(t)

0
00a9
00ae
00f1
0247
0af1
1
10.10
12
120132
14755
160
169
1d310
1f937
2014
20ac
247
255
256
2699
4
437
5
5.1
8.5
8364
9
96
9881
a0000f1
abl
abov
accept
access
activ
ad
addit
advanc
af1
after
all
allow
almost
also
alt
altern
ampersand
ani
anoth
appear
applic
appropri
arbitrari
ascii
assign
associ
augment
autocad
autohotkei
automat
avail
añ
babelmap
back
base
basic
becaus
begin
below
besid
better
block
bmp
both
box
browser
c
call
can
card
case
certain
char
charact
chines
choos
chosen
chromeo
cjk
claus
clipboard
code
codepoint
combin
command
common
comput
confusingli
consum
contain
content
contrast
control
convention
convert
copi
copyright
correspond
coverag
cp1252
creat
ctrl
current
cursor
custom
cut
d
dash
data
decim
defin
depend
describ
desir
desktop
develop
differ
dig
digit
digraph
directli
displai
distinguish
divid
do
document
doe
doubl
down
earlier
edit
editor
either
element
em
emac
emoji
emoticon
empti
emul
enabl
enablehexnumpad
encod
end
engin
enter
entiti
entri
environ
equal
equ

In [ ]:
defaulters = []
for i in tqdm(range(indexReader.numDocs())):
    a = indexReader.document(i)
    try:
        a.get('CONTENT')
    except SystemError as e:
      print("An error occurred:", e)
      defaulters.append(i)


In [ ]:
# print(a.get('CONTENT'))

### document preprocessing

In [ ]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short


def alpha(text):
    a = text.split()
    j = [k for k in a if k.isalpha()]
    return ' '.join(j)

def getprocessdoc(doc_num):
    p = PorterStemmer()
    doc = indexReader.document(doc_num)  
    raw_doc =  doc.get('CONTENT')
    ids = doc.get('ID')

    no_punc = strip_punctuation(raw_doc)
    stemed_doc = p.stem_sentence(no_punc)
    no_stopword = remove_stopwords(stemed_doc)
    no_nonalpha = alpha(no_stopword)
    pro_doc = strip_short(no_nonalpha, minsize=4)

    return pro_doc


In [ ]:
getprocessdoc(1053350)

In [ ]:
def getdoc(num):
    for i in tqdm(range(num)):
        try:
            if i == 1053350:
                pass
            else:
                yield getprocessdoc(i)
        except Exception as e:
            # Handle the exception here, e.g., print an error message
            print(f"Error processing document {i}: {e}")

### tf-idf vectorization using scikitlearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.4, min_df=10)
total_doc = indexReader.numDocs()
X = vectorizer.fit_transform(getdoc(total_doc))

In [ ]:
X

In [ ]:
# with open('wiki_cleaned_doc.vec', 'wb') as fin:
#      pickle.dump(X, fin, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('wiki_cleaned_doc.vec', 'rb') as handle:
    X_tfidf = pickle.load(handle)

In [9]:
X_tfidf

<6584625x889171 sparse matrix of type '<class 'numpy.float64'>'
	with 755238996 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from time import time
import pickle

# with open('wiki_cleaned_doc.vec', 'rb') as handle:
#     X_tfidf = pickle.load(handle)


lsa = make_pipeline(TruncatedSVD(n_components=5), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(X_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

# with open('dim-red-vec.pickle', 'wb') as handle:
#     pickle.dump(X_lsa, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     print('doc-vec saved successfully to file')

In [6]:
X_lsa

array([[ 0.94844218, -0.3169502 ],
       [ 0.89858245, -0.43880472],
       [ 0.8930101 , -0.45003662],
       ...,
       [ 0.98935054, -0.14555246],
       [ 0.90726306, -0.4205636 ],
       [ 0.9961668 , -0.08747401]])

In [ ]:
X_tfidf

In [ ]:
X_lsa

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
true_k = 100
 
# for seed in range(5):
#     kmeans = KMeans(
#         n_clusters=true_k,
#         max_iter=100,
#         n_init=1,
#         random_state=seed,
#     ).fit(X)
#     cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
#     print(f"Number of elements assigned to each cluster: {cluster_sizes}")
# print()


In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
k_range = range(100,200,100)

sse = []

for k in k_range:
    km = KMeans(n_clusters=k)
    km.fit(X_tfidf)
    preds = km.fit_predict(X_tfidf)
    centers = km.cluster_centers_
    score = silhouette_score(X_tfidf, preds)
    print("For n_clusters = {}, silhouette score is {})".format(k, score))
    sse.append(km.inertia_)

In [ ]:
plt.plot(k_range, sse)

In [4]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans, MiniBatchKMeans

# k_range = range(2,20)
k_range = [100]

sse = []

for k in k_range:
    km = MiniBatchKMeans(n_clusters=k, batch_size=6000,
        max_iter=100,
        n_init=5,
        random_state=1, verbose=True).fit(X_tfidf)
   
    preds = km.fit_predict(X_tfidf)

    centers = km.cluster_centers_
    doc_labels = km.labels_
    # score = silhouette_score(X, preds)
    # print("For n_clusters = {}, silhouette score is {})".format(k, score))
    sse.append(km.inertia_)

Init 1/5 with method k-means++
Inertia for init 1/5: 31637.691088999603
Init 2/5 with method k-means++
Inertia for init 2/5: 31642.68711967096
Init 3/5 with method k-means++
Inertia for init 3/5: 31498.58844747782
Init 4/5 with method k-means++
Inertia for init 4/5: 31657.1960511258
Init 5/5 with method k-means++
Inertia for init 5/5: 31448.099160910548
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch step 1/109743: mean batch inertia: 1.750271788897116
[MiniBatchKMeans] Reassigning 2 cluster centers.
Minibatch step 2/109743: mean batch inertia: 0.9564992521069366, ewa inertia: 0.9564992521069366
[MiniBatchKMeans] Reassigning 3 cluster centers.
Minibatch step 3/109743: mean batch inertia: 0.9440116216151838, ewa inertia: 0.9564764943123556
[MiniBatchKMeans] Reassigning 5 cluster centers.
Minibatch step 4/109743: mean batch inertia: 0.940548607817888, ewa inertia: 0.9564474669024565
[MiniBatchKMeans] Reassigning 7 cluster centers.
Minibatch step 5/109743: mean batch inertia: 0

In [ ]:
labels = doc_labels
x_coords = X_lsa[:, 0]
y_coords = X_lsa[:, 1]

# Plot scatter plot with different colors for each cluster
plt.figure(figsize=(8, 6))
plt.scatter(x_coords, y_coords, c=labels, cmap='viridis')
plt.colorbar()

# Add labels and title
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.title('K-means Clustering')

# Show the plot
plt.show()

cluster_ids, cluster_sizes = np.unique(km.labels_, return_counts=True)
print(f"Number of elements id {cluster_ids} \n assigned to each cluster: {cluster_sizes}")